In [16]:
import os
# print(os.getcwd())
os.chdir('D:\OneDrive - zju.edu.cn\Documents\Git\Machine-learning\Machine learning kaggle\IMDB')
print(os.getcwd())

d:\OneDrive - zju.edu.cn\Documents\Git\Machine-learning\Machine learning kaggle\IMDB


In [17]:
import pandas as pd

train_data = pd.read_csv('datasets/labeledTrainData.tsv',delimiter='\t')
test_data = pd.read_csv('datasets/testData.tsv',delimiter='\t')
print(train_data.head())
print(test_data.head())


       id  sentiment                                             review
0  5814_8          1  With all this stuff going down at the moment w...
1  2381_9          1  \The Classic War of the Worlds\" by Timothy Hi...
2  7759_3          0  The film starts with a manager (Nicholas Bell)...
3  3630_4          0  It must be assumed that those who praised this...
4  9495_8          1  Superbly trashy and wondrously unpretentious 8...
         id                                             review
0  12311_10  Naturally in a film who's main themes are of m...
1    8348_2  This movie is a disaster within a disaster fil...
2    5828_4  All in all, this is a movie for kids. We saw i...
3    7186_2  Afraid of the Dark left me with the impression...
4   12128_7  A very accurate depiction of small time mob li...


In [18]:
# review = train_data.loc[0,'review']
# print('review:\n',review)

# stop_words = set(stopwords.words('english'))
# def review_to_text(review,remove_stopwords):
#     raw_text = BeautifulSoup(review,'html').get_text()
#     letters = re.sub('[^a-zA-Z]',' ',raw_text)
#     words = letters.lower().split()
#     print('words:\n',words)
#     if remove_stopwords: 
#         words = [w for w in words if w not in stop_words]
#     print('words:\n',words)
#     return words

# x_train = []
# x_train.append(' '.join(review_to_text(review,True)))
# print('x_train:\n',x_train)

In [20]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
def review_to_text(review,remove_stopwords):
    raw_text = BeautifulSoup(review,'html').get_text()
    letters = re.sub('[^a-zA-Z]',' ',raw_text)
    words = letters.lower().split()
    if remove_stopwords: 
        words = [w for w in words if w not in stop_words]
    return words

x_train = []
for review in train_data['review']:
    x_train.append(' '.join(review_to_text(review,True)))
x_test = []
for review in test_data['review']:
    x_test.append(' '.join(review_to_text(review,True)))

y_train = train_data['sentiment']
# print(x_train[:10])

In [25]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pip_count = Pipeline([('count_vec',CountVectorizer(analyzer='word')),('mnb',MultinomialNB())])
pip_tfidf = Pipeline([('tfidf_vec',TfidfVectorizer(analyzer='word')),('mnb',MultinomialNB())])

params_count = {'count_vec__binary':[True,False],'count_vec__ngram_range':[(1,1),(1,2)],'mnb__alpha':[0.1,1,10.0]}
params_tfidf = {'tfidf_vec__binary':[True,False],'tfidf_vec__ngram_range':[(1,1),(1,2)],'mnb__alpha':[0.1,1,10.0]}

gs_count = GridSearchCV(pip_count,params_count,cv=4,n_jobs=-1,verbose=1)
gs_count.fit(x_train,y_train)
print('gs_count.best_score_:',gs_count.best_score_)
print('gs_count.best_params_:',gs_count.best_params_)
count_y_predict = gs_count.predict(x_test)

gs_tfidf = GridSearchCV(pip_tfidf,params_tfidf,cv=4,n_jobs=-1,verbose=1)
gs_tfidf.fit(x_train,y_train)
print('gs_tfidf.best_score_:',gs_tfidf.best_score_)
print('gs_tfidf.best_params_:',gs_tfidf.best_params_)
tfidf_y_predict = gs_tfidf.predict(x_test)


Fitting 4 folds for each of 12 candidates, totalling 48 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   50.5s finished
gs_count.best_score_: 0.88216
gs_count.best_params_: {'count_vec__binary': True, 'count_vec__ngram_range': (1, 2), 'mnb__alpha': 1}
Fitting 4 folds for each of 12 candidates, totalling 48 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   54.8s finished
gs_tfidf.best_score_: 0.88712
gs_tfidf.best_params_: {'mnb__alpha': 0.1, 'tfidf_vec__binary': True, 'tfidf_vec__ngram_range': (1, 2)}


In [28]:
submission_count = pd.DataFrame({'id':test_data['id'],'sentiment':count_y_predict})
submission_tfidf = pd.DataFrame({'id':test_data['id'],'sentiment':tfidf_y_predict})

submission_count.to_csv('datasets/submission_count.csv',index=False)
submission_tfidf.to_csv('datasets/submission_tfidf.csv',index=False)